In [ ]:
%%capture
import pandas as pd
import nltk
import os
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
from sklearn import feature_extraction
import numpy as np
import heapq
import dotenv
import re
lemmatizer = WordNetLemmatizer()
dotenv.load_dotenv('../ext_variables.env') # Necessary to avoid putting absolute paths
os.chdir(os.getenv("PATH_FILES_ADM"))
tqdm.pandas() # add tqdm progress_apply method for Pandas classes (DataFrame, Series and GroupBy classes)
nltk.download('wordnet') # wordnet
nltk.download("stopwords") # stopword list
nltk.download('punkt') # sentence separation
nltk.download('averaged_perceptron_tagger') # pos-tagging
nltk.download('omw-1.4') # wordnet

In [ ]:
# First of all, let's load the csv/the tsv
places  = pd.read_csv("places.tsv", sep = "\t", index_col=0)
places.drop_duplicates(inplace=True) # Removing duplicates coming from the scraping and crawling process

## 2. Search Engine
<a id = "point_2"></a>

First of all, let's perform some pre-processing with lemmatization (incorporating POS tagging), stopwords removal (what remains of them at that point) and lowercase conversion on the _description_ columns.

The first function in the next block of code converts the [tags from the Penn Treebank project](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) to the ones of [WordNet](https://wordnet.princeton.edu), a powerful word database with a peculiar hierarchical structure. We need the Wordnet tags because the lemmatizer we are using requires that kind of tag.

The second function performs the whole-preprocessing itself:
 - First of all, it tokenizes the strings with `word_tokenize`. This may seem simple, but it is not. Under the hood, `nltk` uses an already trained (for English) unsupervised model able to understand how to split the string into sentences (one can retrain the unsupervised model with a particular corpus, with a specific target language). The output is then parsed with a RegEx expression in order to be split into words.
  - The tokenized output is then passed to the NLTK part-of-speech tagger.
  - The tags are mapped to the ones from WordNet with the function `get_wordnet_pos`, the first defined in the block. Notice that tags not related to adjectives, verbs, nouns or adverbs will be cancelled out by the function, which in that case returns `None`. Once this is done, if the resulting tag has `None` value, the related token/word is not considered.
  - With the tuples `(token, pos_tag)` we can finally call the lemmatizer (more specifically, the _morphy_ function, more info [here](https://wordnet.princeton.edu/documentation/morphy7wn)), and the output is converted to lower case (the output of morphy is lower case, but unsuccessful lemmatization leads to having the unchanged input token returned).
  - If the lemmatization returns a lemma which is not present in any of the Wordnet synsets (groups of words with related meaning) we discard the token.
  - The output is checked against stopwords.
  - The words are joined together with the `|` separator, a character that I do not expect to be popular in the English language.

Notice that with this process we have implicitly removed punctuation.

In [ ]:
# For the following mapping, credits to this stack overflow question:
# https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('RB'):
        return wordnet.ADV
    else:
        return None

# The following is a function which performs the whole pre-processing with lemmatization
# (incorporating information from POS tagging), stopwords removal and lowercase conversion
def preprocessing_field(description: str) -> str:
    description = re.sub(r"(?<=\S)\.(?=\S)", ". ", description) # Add a space after a point where needed in order to enforce proper separation of sentences
    # First of all, we need to tokenize the text.
    # We use word_tokenize from nltk
    tokenized = word_tokenize(description, language='english')
    # Then we use the pos tagger from nltk
    pos_tagged = nltk.pos_tag(tokenized)
    # Then we convert tags from Penn Treebank format to the one of WordNet
    converted_tags_words = [(x[0], get_wordnet_pos(x[1])) for x in pos_tagged]
    # Filter everything that is not an adverb, a verb, a noun or an adjective
    converted_tags_words = [(x[0], x[1]) for x in converted_tags_words if x[1] is not None]
    # Lemmatize the words (morphy function) and convert to lower case
    lemmatized_words = [lemmatizer.lemmatize(word = x[0], pos = x[1]).lower() for x in converted_tags_words]
    # Remove returned lemmas which are not in any of the synsets in WordNet
    lemmatized_words = [x for x in lemmatized_words if wordnet.synsets(x)]
    # Stopwords removal (most of the stopwords were anyway removed with get_wordnet_pos and related filtering)
    # Notice that stopwords removal could also be done later on with CountVectorizer and scikit, but we do it here
    lemmatized_words = [x for x in lemmatized_words if (x not in stopwords.words("english"))]
    # Join the words into strings with words separated explicitly
    # (afterwards we will only need to specify the separator for scikit learn CountVectorizer)
    return "|"+"|".join(lemmatized_words)+"|"

The following is just an example to show what we are doing right now. The text comes from the Wikipedia page of Leo Breiman.

Also notice that, contrary to what we do with stemming, lemmatization retains something which can be understood and read; this in general improves visualization, reporting and debugging with text data. Additionally, lemmatization preserves far more variability than stemming, for obvious reasons (we are reducing words to their dictionary lemma, not to their root), and this may be useful since we are working on a search engine, whose queries may aim at specific words.

However, we may not always want this variability, and this is indeed the case for some specific applications, such as ML modelling where we prefer to catch the signal while disregarding low-level details due to specific lemmas.

In [ ]:
preprocessing_field("Leo Breiman was a distinguished statistician at the University of California, Berkeley. He was the recipient of numerous honors and awards, and was a member of the United States National Academy of Sciences.Breiman's work helped to bridge the gap between statistics and computer science, particularly in the field of machine learning. His most important contributions were his work on classification and regression trees and ensembles of trees fit to bootstrap samples. Bootstrap aggregation was given the name bagging by Breiman. Another of Breiman's ensemble approaches is the random forest.")

Now we apply what we have described to the whole _descriptions_ columns. We are using the integration between `pandas` and `tqdm` in order to keep track of progress. Also notice that we are not overwriting the previous columns (the search engine should return a polished output, not the columns pre-processed for the search engine).

In [ ]:
# Lemmatization of Description and Short Description with POS tagging
# We simply assign the description columns to the transformed descriptions
places["placeDesc_post"], places["placeShortDesc_post"] = places.placeDesc.progress_apply(preprocessing_field), places.placeShortDesc.progress_apply(preprocessing_field)

## 2.1 Conjunctive query
<a id = "point_2.1"></a>

Next step is building the inverted index. In order to do that, the first step is building a so-called Term-Document matrix, which is easily built with scikit-learn, more specifically with its (very, very useful) `CountVectorizer` class. The `transform` method of the class returns a Document-Term matrix, so we need to take its transpose. For every row (a document) we will have a list (along the columns) of one-hot representations (presence or absence of a word).

We need to pass a RegEx expression to enforce the separator we have placed before: `|`. We also pass other arguments to the `__init__` method, but they are relatively straightforward.


In [ ]:
# The RegEx token pattern is very simple here, it just has a single capturing group with a non-greedy quantifier and a lookahead and a lookbehind to avoid consuming "|" in the match, which is a NO-GO
# Binary = TRUE => one hot encoding representation
one_hot_vectorizer = feature_extraction.text.CountVectorizer(strip_accents=False, lowercase=False, token_pattern=r"(?<=\|)(.*?)(?=\|)", binary = True)
one_hot_vectorizer.fit(places.placeDesc_post) # Get the vocabulary from the corpus
term_document = one_hot_vectorizer.transform(places.placeDesc_post).transpose() # Transform the corpus into the document-term matrix, then take the transpose of the output

We then get the dictionary mapping each word to its index in the matrix. In order to do that, we can just access an attribute of the one_hot_vectorizer object. It will be saved, serialized, in storage. For that we can just use the `pickle` Python module.

In [ ]:
import pickle
# The vocabulary is saved, serialized, in storage
vocabulary_word_index = one_hot_vectorizer.vocabulary_
with open("vocabulary_word_index.pickle", "wb") as vocab_file:
    pickle.dump(vocabulary_word_index, file = vocab_file)

In order to get the dictionary/the inverted index we just need to extract the indexes of the non-zero entries in the sparse Document-Term matrix. Since the `sparse.csc_matrix` class of _scipy_ (whose instance is returned by the `transform` method of the `CountVectorizer` class) has a method for this, we can just use that one. What is returned, as usual, are two NumPy arrays, one for the indexes referring to the rows, and one for the indexes referring to the columns. They are two iterables, so we `zip` them together and iterate over them in order to build the inverted index.

Again, the resulting dictionary is saved in storage as a serialized object with the `pickle` module.

In [ ]:
from collections import defaultdict
inverted_index_onehot = defaultdict(list)
for row in zip(term_document.nonzero()[0], term_document.nonzero()[1]):  # Build iterator that returns row and column index of non zero at each iteration
    inverted_index_onehot[row[0]].append(row[1])
with open("inverted_index_onehot.pickle", "wb") as inverted_index_file:
    pickle.dump(inverted_index_onehot, file = inverted_index_file)

In order to load them into memory from storage we simply need to use `pickle` again, this time with the `load` function.

In [ ]:
import pickle

with open("vocabulary_word_index.pickle", "rb") as vocab_file:
    vocabulary_word_index = pickle.load(vocab_file)
with open("inverted_index_onehot.pickle", "rb") as inverted_index_file:
    inverted_index_onehot = pickle.load(inverted_index_file)

At this point, we just need to define the search engine function. It is enough to take the intersection between the sets/the lists (the values in the dictionary) containing the document ids for each of the terms in order to get the output of the search engine from the original DataFrame.

In [ ]:
import re
def search_engine1(vocabulary:dict[str:int], inverted_index:dict[int: list[int, ...]], dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    This function asks for a query and retrieves the rows from the input DataFrame whose pre-processed placeDesc entry contains all the words in the query.
    It does this by completely relying on an inverted index data structure, which in turn relies on a vocabulary dictionary in order to be mapped back to the
    original string representations of the words/tokens.
    :param vocabulary: The vocabulary dictionary mapping the string representation for each token to the related integer index
    :param inverted_index: The inverted index mapping each index to a list of documents ids
    :param dataframe: The Pandas DataFrame from which to retrieve the places. Notice that the function assumes that the Dataframe has [placeName, placeURL and placeDesc] in its column index
    :return: a Pandas DataFrame with all the documents containing all the words in the query
    """
    query = input("Query: ").strip() # Ask for input (removing leading and trailing whitespace)
    query_elements = re.split(r'\s+', query.lower()) # Split according to one or more whitespace with a simple expression
    query_elements = list(set(query_elements)) # Get only unique words
    token_ids = list(map(vocabulary.get, query_elements)) # Get the ids for each of the tokens from the vocabulary. None is returned if requested token is not present. Note that we have to get a list from the map object; this is due to the fact that the returned map object is an iterator, and thus the __iter__ method returns the object itself.
    output_docs = []
    for id in token_ids:
        # If a term is missing in the vocabulary, the loop is immediately stopped, and the DataFrame equivalent of None is returned by passing an empty list to Pandas indexing.
        if not all(token_ids):
            break
        if not output_docs: # Output docs is empty list at first, and it evaluates to False
            output_docs = set(inverted_index[id]) # Initialize the set of the output docs (1st term)
        else:
            output_docs = output_docs.intersection(set(inverted_index[id])) # Take the intersection for each term after the first
    output = dataframe.iloc[list(output_docs)][["placeName", "placeDesc", "placeURL"]]
    output.placeURL = output.placeURL.str.replace(r"https://www.atlasobscura.com", "", regex = False) # Remove redundant part of the URL
    return output

In [ ]:
print('Result for \'American Museum\':')
search_engine1(vocabulary_word_index, inverted_index_onehot, places)

## Conjunctive query & ranking score
<a id = "point_2.2"></a>

The first step here is similar if not identical to the one for [Point 2.1](#point_2.1). We use again scikit-learn, but this time with the TF-IDF Vectorizer. We already have the dictionary, so we pass it to the constructor method of the class. Notice that the computed sparse document term matrix has its row vectors already L2-normalized. This makes our lives easier when computing the cosine similarity later on, as we will explain.

The idf definition used by scikit when `smooth_idf=False` (we do not need the smoothing here, since the vocabulary is built directly from the corpus) is the following (can be checked by looking at the [documentation page](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer) of the `TfidfTransformer` class):
$$
idf(t) = \log\left(\frac{N}{df(t)}\right) + 1
$$

This means that the baseline for the tf-idf is 1 and not 0.

In [ ]:
# The RegEx token pattern is very simple here, it just has a single capturing group with a non-greedy quantifier and a lookahead and a lookbehind to avoid consuming "|" in the match, which is a NO-GO
# Binary = TRUE => one hot encoding representation
tfidf_vectorizer = feature_extraction.text.TfidfVectorizer(strip_accents=False, lowercase=False, token_pattern=r"(?<=\|)(.*?)(?=\|)", vocabulary = vocabulary_word_index, smooth_idf=False)
tfidf_vectorizer.fit(places.placeDesc_post)
term_document_tfidf = tfidf_vectorizer.transform(places.placeDesc_post).transpose() # Transform the corpus into the document-term matrix (with tf-idf weights), then take the transpose of the output

Now we have to build the new inverted index with the tf-idf information. The way we build it is very similar to what we have done in [Point 2.1](#Point_2.1). The main difference here is that we need to incorporate the tf-idf information, with the value for each key in the dictionary becoming a list of tuples.

First of all, we need to access the non-zero elements of the sparse matrix returned by the `transform` method of the `TfidfVectorizer` class. We can just use some fancy indexing on the sparse matrix object to do that.

In [ ]:
# %%prun # Profile the code to check that fancy indexing is working (documentation for SciPy is really not up to the standards)
# I wanted to be sure that it was NOT iterating over the indexes in order to retrieve the elements from the sparse matrix. In other words, I wanted to check the vectorization of the retrieval op
from collections import defaultdict
inverted_index_tfidf = defaultdict(list)

nonzero_entries = term_document_tfidf.nonzero() # Get indexes (for rows and for columns) of the non-zero entries in the sparse matrix

flattened_sparse = np.asarray(term_document_tfidf[nonzero_entries[0], nonzero_entries[1]]).flatten() # Flatten the matrix to a vector of its non-zero entries (simple fancy indexing op)
for row in zip(nonzero_entries[0], nonzero_entries[1], flattened_sparse):  # Build iterator that returns row and column index + value of non-zero entries at each iteration
    inverted_index_tfidf[row[0]].append((row[1], row[2])) # Appending tuples to the list assigned to each term/token/key
with open("inverted_index_tfidf.pickle", "wb") as inverted_index_file:
    pickle.dump(inverted_index_tfidf, file = inverted_index_file)

For this second search engine, we need to build upon the first one by considering the cosine similarity between the query itself and the tf-idf representation of the documents. Since we do expect users to only write short queries with unique words, it makes sense to consider the _angle_ given by the one-hot encoding of the unique words in the query, without highlighting them considering their frequency in the corpus (through the idf score).

The reasoning is the following: if a user writes some words for the query, they think that those words are equally relevant for the search, so it does not make sense to give them a differentiated weight in the query representation; in the documents, those frequent words are relatively penalized w.r.t. infrequent ones, and it makes sense that they are, since the editors of a document write according to a syntactic and semantic flow, and common words may be common just because they are common in a language and in the context of the corpus.

Therefore, in order for the document vectors to be more or less parallel to the query representation, they need to have **balanced** tf-idf entries for all the words present in the query. A query with both a common and a rare word returns a high cosine similarity if the document vector has more occurrences of the common word than it has for the rarer word; this is due to the fact that the common word is very meaningful in the query and not meaningful at all in the document, so the term frequency at document level needs to compensate that.

For the following implementation, please also notice another thing: as we said above, the tf-idf representations are already **L2 normalized**, and this means that the cosine similarity is easier to compute. With $\textbf{q}$ as the one-hot encoded representation of the query and $\textbf{d}$ as the normalized tf-idf representation of the document, the cosine similarity is this way defined (due to the L2 normalization):

$$
\theta = \frac{\textbf{q}\cdot\textbf{d}}{||\textbf{q}||_2||\textbf{d}||_2} = \frac{\textbf{q}\cdot\textbf{d}}{||\textbf{q}||_2}, ||\textbf{d}||_2 = 1
$$

In our implementation, this can be simplified to dividing the sum of the entries in the tf-idf document vectors that are related to the unique words present in the query by the square root of the number of unique words in the query. It is trivial to show this, given that the query representation is a one-hot encoded vector.

In [ ]:
def search_engine2(vocabulary:dict[str:int], inverted_index:dict[int: list[int, ...]], dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    This function asks for a query and retrieves the 5 top rows/places from the input DataFrame whose pre-processed placeDesc entry contains all the words in the query.
    The 5 top places are extracted by using a max-heap and they are defined according to the cosine similarity of the placeDesc entry/document with the query.
    The pipeline completely relies on an inverted index data structure, which in turn relies on a vocabulary dictionary in order to be mapped back to the original string
    representations of the words/tokens.
    :param vocabulary: The vocabulary dictionary mapping the string representation for each token to the related integer index
    :param inverted_index: The inverted index mapping each index to a list of tuples, each containing a documents id and a related tf-idf value
    :param dataframe: The Pandas DataFrame from which to retrieve the places. Notice that the function assumes that the Dataframe has [placeName, placeURL and placeDesc] in its column index
    :return: a Pandas DataFrame with the top 5 documents according to cosine similarity with the query
    """
    query = input("Query: ").strip() # Ask for input (removing leading and trailing whitespace)
    query_elements = re.split(r'\s+', query.lower()) # Split according to one or more whitespace with a simple expression
    query_elements = list(set(query_elements)) # Get only unique words
    output_docs = None
    token_ids = list(map(vocabulary.get, query_elements)) # Map each of the tokens to their ids/values in the dictionary. None is returned if not present. None is falsy


    for id in token_ids:
        # If a term is missing in the vocabulary, the function is immediately stopped, and the DataFrame equivalent of None is returned by passing an empty list to Pandas indexing
        if not all(token_ids):
            return dataframe.iloc[[]][["placeName", "placeDesc", "placeURL"]]
        if not output_docs: # Output docs is None at first, and it evaluates to False
            output_docs = set([x[0] for x in inverted_index[id]]) # Initialize the set of the output docs (1st term)
        else:
            output_docs = output_docs.intersection(set([x[0] for x in inverted_index_tfidf[id]])) # Take the intersection for each term after the first


    # Now we need to sort according to TF-IDF and cosine similarity with the query
    output_docs = list(output_docs) # We need an ordered structure, set has no order
    output_array = np.empty(shape=(len(token_ids), len(output_docs)), dtype=np.float64) # Allocate array which will contain the tfidf values for each of the query token for each of the output documents

    for iter_index, id in enumerate(token_ids):
        tf_idf_dict = dict(inverted_index_tfidf[id]) # When we pass a list of tuples (such as in this case) to the dictionary constructor, they are considered key:values pairs
        output_array[iter_index] = list(map(tf_idf_dict.get, output_docs)) # each row contains the tf_idf values (one for each output doc) for a specific token


    # We are doing a lot in the following row. First of all we are computing the cosine similarity between each of the docs and the query (refer to the description above this function
    # definition to understand the reasoning). The second thing we do is creating an iterator of tuples (cosine similarity and document ids) with zip. Then we exhaust that iterator and move
    # its elements into a list
    heap_output = list(zip(-output_array.sum(0)/np.sqrt(output_array.shape[0]), output_docs))
    # Change the order of the elements of the list (in place) in order to make it represent a heap. The result is a max heap because we have changed the sign of the entries heapify returns an array for a min heap
    heapq.heapify(heap_output)


    index_df = [] # Initialize list containing indexes of the documents/DataFrame rows
    cosine_sim = [] # Initialize list containing cosine similarity
    for i in range(min(5, len(heap_output))):
        doc = heapq.heappop(heap_output) # Repeated heappop after heapify is exactly equivalent to heapsort
        index_df.append(doc[1])
        cosine_sim.append(-doc[0])

    output_df = dataframe.iloc[index_df][["placeName", "placeDesc", "placeURL"]]
    output_df["cosine_sim"] = cosine_sim
    output_df.placeURL = output_df.placeURL.str.replace(r"https://www.atlasobscura.com", "", regex = False) # Remove redundant part of the URL

    return output_df

In [ ]:
print('Result for \'American Museum\':')
search_engine2(vocabulary_word_index, inverted_index_tfidf, places)

## 3. Define a new score!
<a id="point3"></a>

In this case we have to compute a new score in order to build a new search engine. More things need to be considered:
- First thing that comes to mind is considering the name of the place itself.
- Second thing in a ranking of importance is the country the user is aiming at.
- Third thing, in order to distinguish between popular and niche places, we need to consider both the number of people that want to visit and the number of people that have visited the places.

In order to build the new ranking system/the new score, we can ask the user to input additional information after the starting query, and then use that in order to do some additional manipulations and numerical operations. Let's analyze in depth what we are doing here:
- The name of the place is factored in by asking the user for some words he may remember and/or search from the name of the place. For all the $N_t$ input words, if the name of the place contains one of the words: $n_{t, i} \leftarrow n_{t, i}+1$, with $n_{t, i}\leftarrow 0 $ for initialization. The final score for this point is given by $t_i = \frac{n_{t, i}}{N_t} $. Notice that we obtain something bounded between 0 and 1.
- The part of the score related to the country is way easier. $c_i = 1 \iff place_i \in CountryQueried$. To compute this we can just look at the last part of the `placeAddress` field of the original DataFrame.
- Scoring according to popularity is a bit more sophisticated. We are asking for 1 to 3 as popularity, and this translates back to intervals stemming from the percentiles of the (complete) sample distribution of the `numPeopleWant` `numPeopleVisited` columns. 1 as popularity means querying for something in the 33 percentile, 2 as popularity means querying for something between the 33 percentile and the 66 percentile and 3 as popularity means residing above the 66 percentile. From checking this we obtain two booleans (one for `numPeopleWant` and one for `numPeopleVisited`), and by taking their mean (True is 1, False is 0) we obtain something bounded between 0 and 1. This score is $p_i$.

The final value for the ith place is given, with $cs_i$ as the cosine similarity, by: $s_i = 0.1cs_i + 0.3t_i + 0.3c_i + 0.3p_i$. This can be seen as a weighted average between the scores, with the cosine similarity being given the lowest score due to the fact the information regarding the description entry is already used in order to build the list of places to score in the first place.


In [ ]:
def search_engine3(vocabulary:dict[str:int], inverted_index:dict[int: list[int, ...]], dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    This function asks for a query and 3 additional pieces of information, regarding the name of the place(s), the country of the places and the popularity of the places.
    It first retrieves the places/rows from the input DataFrame whose pre-processed placeDesc entry contains all the words in the query.
    Each resulting place is scored according to the cosine similarity of the placeDesc(ription) entry with the starting query, the number of input words in the name of the place,
    the match with the requested country and the match with the requested popularity. 5 top places are extracted by using a max-heap data structure (in array representation, a list)
    on the score and by repeatedly popping the root of the heap (heapsort).
    :param vocabulary: The vocabulary dictionary mapping the string representation for each token to the related integer index
    :param inverted_index: The inverted index mapping each index to a list of tuples, each containing a documents id and a related tf-idf value
    :param dataframe: The Pandas DataFrame from which to retrieve the places. Notice that the function assumes that the Dataframe has [placeName, placeURL and placeDesc] in its column index
    :return: a Pandas DataFrame with the top 5 documents according to the new score
    """

    query = input("Query: ").strip()

    score_att = []
    for i in range(3):
        if i == 0:
            score_att.append(input("Words from the name of the place: ").strip())
        elif i == 1:
            score_att.append(input('Countries, comma separated: ').strip())
        elif i == 2:
            score_att.append(input("How much popular, from 1 to 3: ").strip())

    # Get documents where at least one of the words specified in the query is present

    query_elements = re.split(r'\s+', query.lower()) # Split according to one or more whitespace with a simple expression
    query_elements = list(set(query_elements)) # Get only unique words
    output_docs = None
    token_ids = list(map(vocabulary.get, query_elements)) # Map each of the tokens to their ids/values in the dictionary. None is returned if not present. None is falsy


    for id in token_ids:
        # If a term is missing in the vocabulary, the function is immediately stopped, and the DataFrame equivalent of None is returned by passing an empty list to Pandas indexing
        if not all(token_ids):
            return dataframe.iloc[[]][["placeName", "placeDesc", "placeURL"]]
        if not output_docs: # Output docs is None at first, and it evaluates to False
            output_docs = set([x[0] for x in inverted_index[id]]) # Initialize the set of the output docs (1st term)
        else:
            output_docs = output_docs.intersection(set([x[0] for x in inverted_index_tfidf[id]])) # Take the intersection for each term after the first

    output_docs = list(output_docs) # We need an ordered structure, set has no order. These are the documents/the observations for which we have to compute the score

    total_score = np.zeros(len(output_docs), dtype=np.float64) # Initialize array containing overall scores

    # Cosine similarity part of the score computation
    output_array = np.empty(shape=(len(token_ids), len(output_docs)), dtype=np.float64) # Allocate array which will contain the tfidf values for each of the query token for each of the output documents
    for iter_index, id in enumerate(token_ids):
        tf_idf_dict = dict(inverted_index_tfidf[id])
        output_array[iter_index] = list(map(tf_idf_dict.get, output_docs)) # each row contains the tf_idf values (one for each output doc) for a specific token


    # The content of this line is commented in depth in the definition of the search engine for point 2.2
    total_score += 0.1 * np.squeeze(np.array(list(zip(output_array.sum(0)/np.sqrt(output_array.shape[0])))), 1) # update the total score (now a zero vector), element-wise

    docs_df = dataframe.iloc[output_docs] # This is needed to access the other fields without continuously calling iloc with the doc ids

    # Name of the place part of the score computation
    if score_att[0]:
        name_pieces = re.split(r'\s+', score_att[0]) # Split the string containing name words
        name_array = np.empty(shape=(len(name_pieces), len(docs_df)), dtype=np.bool_) # Initialize array of boolean (the documents have or not the specific word? len(name_pieces) words, len(docs_df) documents
        for iter_index, piece in enumerate(name_pieces):
            name_array[iter_index] = docs_df.placeName.str.contains(rf'\s{piece}\s', regex=True, flags=re.IGNORECASE) # assign a row to boolean 1D array coming from Pandas string op that checks condition
        total_score += 0.3*name_array.mean(axis=0) # update the total score, element-wise with mean over boolean fields (1s and 0s)


    # Country part of the score computation
    if score_att[1]:
        countries = '|'.join([x.strip() for x in score_att[1].split(',')]) # Build part of RegEx pattern to check
        total_score += 0.3 * docs_df.placeAdress.str.match(rf'.*,\s({countries})$', flags=re.IGNORECASE) # Complete and use RegEx pattern to check country

    # Popularity part of the score computation
    if score_att[2]:
        percentile_interval = ((1/3)*(float(score_att[2])-1), (1/3)*float(score_att[2])) # Get the interval we are aiming at
        # The score is simply the mean between the two boolean fields (1s and 0s) which are obtained by checking if the observation is in the required interval for the two features
        pop_score = np.array((docs_df[["numPeopleVisited", "numPeopleWant"]] > places[['numPeopleVisited', 'numPeopleWant']].quantile(percentile_interval[0], interpolation = 'higher')) & (docs_df[["numPeopleVisited", "numPeopleWant"]] < places[['numPeopleVisited', 'numPeopleWant']].quantile(percentile_interval[1], interpolation = 'higher'))).mean(axis = 1)
        total_score += 0.3 * pop_score # Update total score element-wise

    # Build the heap from the scores + the ids
    heap_output = list(zip(-total_score, output_docs)) # Build a list from zipped total scores and output docs
    heapq.heapify(heap_output) # Heapify ---> build max heap

    output_id = []
    output_ordered_scores = []
    for i in range(min(5, len(heap_output))): # Heap
        popped = heapq.heappop(heap_output) # Heapsort
        output_id.append(popped[1])
        output_ordered_scores.append(-popped[0])

    output_df = places.iloc[output_id][["placeName", "placeDesc", "placeURL", "placeAdress", "placeTags", "numPeopleVisited", "numPeopleWant", "placeAlt", "placeLong"]]
    output_df.insert(3, "new_score", output_ordered_scores)
    output_df.placeURL = output_df.placeURL.str.replace(r"https://www.atlasobscura.com", "", regex = False) # Remove redundant part of the URL
    return output_df

In [ ]:
third_output = search_engine3(vocabulary_word_index, inverted_index_tfidf, places)

In [ ]:
print("Output for American Museum + \'medical\' + \'Thailand, United States, France\' + 1 as popularity: ")
third_output

Notice that without any additional information (w.r.t. the initial query) the score is the exact same of what was done in [Point 2.2](#point2.2), just with a scaled cosine similarity. A user thus could easily revert to the previous search engine. That said, I think that this third engine works better since it allows the users to refine the queries and thus get specific places highlighted according to features he may remember and/or search for. With more information, auspiciously, we should get better results.

As an example of this new scoring system, let's look at the _Siriraj Medical Museum case_, which appeared as the first result in the second search engine. In this case, it is still first, but just because the additional features we have passed to the engine actually match the place. Let us go through it together. _American Museum_ is the same query as always, so the cosine similarity is the same as before, $+0.354277\cdot0.1$; _medical_ is contained in the title, so $+ 0.3$; _Thailand_, the country the place is in, is in the countries we have passed to the engine, so $+ 0.3$; and then $+0.15$ for the popularity since we have passed 1 and the place is above the 33 percentile for the `numPeopleVisited` field and below it for the `numPeopleWant` field. Summing everything up we obtain 0.7854277, which, approximated to the 6th decimal, is exactly 0.785428, the value we have in the table.

## 4. Visualizing the most relevant places

In order to visualize the most relevant places we can use interactive maps with **Plotly**. A **bubble plot** may be useful in order to understand the number of people who have visited each place, while for everything else we should exploit the interactivity of the plot.

As one can see in the code, we have indeed pushed the information concerning the address and the name of the place (required for the exercise) in the text boxes popping up when hovering over the point/the place on the map. The relative size of the marker increases as the number of people that have visited the place grows. That piece of information is thus conveyed visually.

The country boundaries should be enough to understand which country the place is in, but for what concerns address and name there was no cleaner way than to use hover text. Also notice that the initial zoom for the map is automatically selected in order for it to be just enough to capture all the points in the window.


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Set the overall plot, with red as color and triangle as shape for the markers. The size of the markers is given by the number of people that have visited the place
# the fitbounds argument automatically zooms the map
fig_places = px.scatter_geo(third_output, lat = 'placeAlt', lon = 'placeLong', color_discrete_sequence = ['red'], symbol_sequence = ['triangle-up-dot'], opacity = 0.9, width = 1300, height=700, size = 'numPeopleVisited', fitbounds='locations')

# Add title and subtitle (we can use HTML in Plotly)
fig_places.update_layout(title=go.layout.Title(text = 'Interactive geospatial visualization of the 5 most relevant places according to user query<br>'
                                               '<sup>Size of the marker grows with the number of '
                                               'people that have visited the place. Hovering over the points returns additional information.</sup>',
                                               xref="paper", font=dict(size=25, family='Droid Serif')))

# Add country borders with black color, change color of the lands (I am a bit color-blind, so don't judge XD) and remove lakes from visualization
fig_places.update_geos(showcountries=True, countrycolor = 'black', landcolor='tan', showlakes=False)

# Set the text for each text box popping up when hovering over a point
fig_places.update_traces(customdata = third_output[["placeAdress", "placeName", "numPeopleVisited", "numPeopleWant", "numPeopleWant"]],
                  hovertemplate='%{customdata[1]}<br>Address: %{customdata[0]}<br>Number of people that have visited the place: %{customdata[2]}<br>')

fig_places.show()